In [ ]:
!pip install textacy

In [ ]:
import sys, os
ON_COLAB = 'google.colab' in sys.modules

if ON_COLAB:
    os.system("test -f heise-articles-2020.db || wget  https://datanizing.com/heiseacademy/nlp-course/blob/main/99_Common/heise-articles-2020.db.gz && gunzip heise-articles-2020.db.gz")
    newsticker_db = 'heise-articles-2020.db'
else:
    newsticker_db = '../99_Common/heise-articles-2020.db'

In [ ]:
import sqlite3 
import pandas as pd

sql = sqlite3.connect(newsticker_db)
df = pd.read_sql("SELECT * FROM nlp_articles ORDER BY datePublished", sql, index_col="id")

In [ ]:
import regex as re

def single_words(df, field):
    return [w for words in df[field] for w in re.split(r'\||\#', words)]

In [ ]:
from collections import Counter

nouns = Counter(single_words(df, "nouns"))
adjectives = Counter(single_words(df, "adjectives"))
verbs = Counter(single_words(df, "verbs"))
nav = Counter(single_words(df, "nav"))
lemmas = Counter(single_words(df, "lemmas"))

In [ ]:
noun_chunks = Counter(single_words(df, "noun_chunks"))
entities = Counter(single_words(df, "entities"))

In [ ]:
nouns.most_common()

In [ ]:
!pip install wordcloud

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt

wc = WordCloud(background_color="white", max_words=100)
wc.generate_from_frequencies(nouns)
plt.figure(figsize=(12,12))
plt.imshow(wc, interpolation='bilinear')
plt.axis("off");

In [ ]:
wc = WordCloud(background_color="white", max_words=100)
wc.generate_from_frequencies(verbs)
plt.figure(figsize=(12,12))
plt.imshow(wc, interpolation='bilinear')
plt.axis("off");

In [ ]:
wc = WordCloud(background_color="white", max_words=100)
wc.generate_from_frequencies(adjectives)
plt.figure(figsize=(12,12))
plt.imshow(wc, interpolation='bilinear')
plt.axis("off");

In [ ]:
wc = WordCloud(background_color="white", max_words=100)
wc.generate_from_frequencies(nav)
plt.figure(figsize=(12,12))
plt.imshow(wc, interpolation='bilinear')
plt.axis("off");

In [ ]:
from spacy.lang.de.stop_words import STOP_WORDS
for w in STOP_WORDS:
    nav[w] = 0

In [ ]:
wc = WordCloud(background_color="white", max_words=100)
wc.generate_from_frequencies(nav)
plt.figure(figsize=(12,12))
plt.imshow(wc, interpolation='bilinear')
plt.axis("off");

In [ ]:
wc = WordCloud(background_color="white", max_words=100)
wc.generate_from_frequencies(noun_chunks)
plt.figure(figsize=(12,12))
plt.imshow(wc, interpolation='bilinear')
plt.axis("off");

In [ ]:
wc = WordCloud(background_color="white", max_words=100)
wc.generate_from_frequencies(entities)
plt.figure(figsize=(12,12))
plt.imshow(wc, interpolation='bilinear')
plt.axis("off");